## Masked Language Modeling
Using MLM, we train adapters for each of the GLUE tasks. This adapts the pre-trained language model to the language corpus specific to the GLUE task

In [1]:
from mlm import masked_language_modeling
from mlm_utils import DomainModelArguments, DomainDataTrainingArguments
from transformers import TrainingArguments, MultiLingAdapterArguments

In [2]:
glue_tasks = [
    "cola",
    "mnli",
    #"mrpc",
    "qnli",
    "qqp",
    "rte",
    "sst2",
    "stsb",
    "wnli",
]

In [3]:
model = DomainModelArguments(
    model_name_or_path="roberta-base",
)

adapter = MultiLingAdapterArguments(
    train_adapter=True,
    adapter_config="pfeiffer+inv",
)

In [4]:
# %%capture
results = {}
for dataset in glue_tasks:
    data = DomainDataTrainingArguments(
        dataset_name="glue",
        dataset_config_name=dataset,
    )
    
    training = TrainingArguments(
        learning_rate=1e-4,
        overwrite_output_dir=True,
        output_dir=f"./adapter/mlm/{dataset}",
        do_train=True,
        do_eval=True,
        num_train_epochs=10,
    )

    train_stats, eval_stats = masked_language_modeling(
        model_args=model, data_args=data, training_args=training, adapter_args=adapter
    )
    
    results[dataset] = {"training" : train_stats, "eval" : eval_stats}


07/26/2021 12:41:49 - WARNING - mlm -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2021 12:41:49 - INFO - mlm -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=0.0001,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=./adapter/mlm/col

[INFO|configuration_utils.py:531] 2021-07-26 12:41:50,365 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/jason/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:569] 2021-07-26 12:41:50,368 >> Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "t

07/26/2021 12:41:52 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /home/jason/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ec10cef38504e666.arrow
07/26/2021 12:41:52 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /home/jason/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-88a93ce67a4e181d.arrow
07/26/2021 12:41:52 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /home/jason/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-231f91ea1ab5bd4f.arrow
07/26/2021 12:41:52 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /home/jason/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-cf457f7d064846e8.arrow
07/26/2021 12:41:52 

[INFO|trainer.py:546] 2021-07-26 12:41:54,411 >> The following columns in the training set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
[INFO|trainer.py:1199] 2021-07-26 12:41:54,416 >> ***** Running training *****
[INFO|trainer.py:1200] 2021-07-26 12:41:54,416 >>   Num examples = 185
[INFO|trainer.py:1201] 2021-07-26 12:41:54,416 >>   Num Epochs = 10
[INFO|trainer.py:1202] 2021-07-26 12:41:54,416 >>   Instantaneous batch size per device = 8
[INFO|trainer.py:1203] 2021-07-26 12:41:54,416 >>   Total train batch size (w. parallel, distributed & accumulation) = 8
[INFO|trainer.py:1204] 2021-07-26 12:41:54,417 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:1205] 2021-07-26 12:41:54,417 >>   Total optimization steps = 240


Step,Training Loss


...

Step,Training Loss


[INFO|trainer.py:1403] 2021-07-26 14:41:30,664 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


[INFO|trainer.py:1989] 2021-07-26 14:41:30,665 >> Saving model checkpoint to ./adapter/mlm/wnli
[INFO|loading.py:59] 2021-07-26 14:41:30,666 >> Configuration saved in ./adapter/mlm/wnli/glue/adapter_config.json
[INFO|loading.py:72] 2021-07-26 14:41:30,673 >> Module weights saved in ./adapter/mlm/wnli/glue/pytorch_adapter.bin
[INFO|loading.py:59] 2021-07-26 14:41:30,674 >> Configuration saved in ./adapter/mlm/wnli/glue/head_config.json
[INFO|loading.py:72] 2021-07-26 14:41:30,824 >> Module weights saved in ./adapter/mlm/wnli/glue/pytorch_model_head.bin
[INFO|loading.py:59] 2021-07-26 14:41:30,825 >> Configuration saved in ./adapter/mlm/wnli/glue/head_config.json
[INFO|loading.py:72] 2021-07-26 14:41:31,018 >> Module weights saved in ./adapter/mlm/wnli/glue/pytorch_model_head.bin
[INFO|tokenization_utils_base.py:1948] 2021-07-26 14:41:31,018 >> tokenizer

***** train metrics *****
  epoch                    =       10.0
  total_flos               =   111652GF
  train_loss               =     1.6329
  train_runtime            = 0:00:06.11
  train_samples            =         31
  train_samples_per_second =     50.686
  train_steps_per_second   =       6.54
07/26/2021 14:41:31 - INFO - mlm -   *** Evaluate ***


[INFO|trainer.py:546] 2021-07-26 14:41:31,069 >> The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
[INFO|trainer.py:2239] 2021-07-26 14:41:31,071 >> ***** Running Evaluation *****
[INFO|trainer.py:2241] 2021-07-26 14:41:31,071 >>   Num examples = 3
[INFO|trainer.py:2244] 2021-07-26 14:41:31,071 >>   Batch size = 8


***** eval metrics *****
  epoch                   =       10.0
  eval_loss               =     1.6839
  eval_runtime            = 0:00:00.02
  eval_samples            =          3
  eval_samples_per_second =    105.691
  eval_steps_per_second   =      35.23
  perplexity              =     5.3866


In [5]:
from pprint import pprint
pprint(results)

{'cola': {'eval': {'epoch': 10.0,
                   'eval_loss': 1.679701566696167,
                   'eval_runtime': 0.1956,
                   'eval_samples': 22,
                   'eval_samples_per_second': 112.484,
                   'eval_steps_per_second': 15.339,
                   'perplexity': 5.3639549494375895},
          'training': {'epoch': 10.0,
                       'total_flos': 715446823680000.0,
                       'train_loss': 1.250021235148112,
                       'train_runtime': 36.9675,
                       'train_samples': 185,
                       'train_samples_per_second': 50.044,
                       'train_steps_per_second': 6.492}},
 'mnli': {'eval': {'epoch': 10.0,
                   'eval_loss': 1.854617953300476,
                   'eval_runtime': 8.4033,
                   'eval_samples': 994,
                   'eval_samples_per_second': 118.286,
                   'eval_steps_per_second': 14.875,
                   'perplexity': 6.3